In [4]:
# Import libraries
import torch

In [9]:
# Define non-maximum suppression function
def nms_pytorch(P : torch.tensor, threshIoU : float):
    # Extract coordinates for every prediction box present in P
    x1 = P[:, 0]
    y1 = P[:, 1]
    x2 = P[:, 2]
    y2 = P[:, 3]
    
    # Extract the confidence score as well
    scores = P[:, 4]
    
    # Calculate the area of every bounding box in P
    areas = (x2 - x1) * (y2 - y1)
    
    # Sort the bounding boxes in P according to their confidence scores
    order = scores.argsort()
    
    # Initialize an empty list to keep filtered bounding boxes
    keep = []
    
    while len(order) > 0:
        # Extract the index of the bounding box with highest confidence score
        idx = order[-1]
        
        # Append that bounding box to filtered predictions list
        keep.append(P[idx])
        
        # Remove that bouding box from P
        order = order[:-1]
        
        # Select coordinates of bounding boxes according to the indices in order
        xx1 = torch.index_select(x1, dim=0, index=order)
        xx2 = torch.index_select(x2, dim=0, index=order)
        yy1 = torch.index_select(y1, dim=0, index=order)
        yy2 = torch.index_select(y2, dim=0, index=order)
        
        # Find the coordinates of the intersection boxes
        xx1 = torch.max(xx1, x1[idx])
        yy1 = torch.max(yy1, y1[idx])
        xx2 = torch.min(xx2, x2[idx])
        yy2 = torch.min(yy2, y2[idx])
        
        # Find the height and width of the intersection boxes
        w = xx2 - xx1
        h = yy2 - yy1
        
        # Take max with 0.0 to avoid negative w and h due to non-overlapping boxes
        w = torch.clamp(w, min=0.0)
        h = torch.clamp(h, min=0.0)
        
        # Find the area of the intersection boxes
        inter = w * h
        
        # Find the areas of bounding boxes according to the indices in order
        remAreas = torch.index_select(areas, dim=0, index=order)
        
        # Find the union of every prediction in P with the prediction that has the highest confidence score
        union = (remAreas - inter) + areas[idx]
        
        # Find the IoU of every prediction in P with the prediction that has the highest confidence score
        IoU = inter / union
        
        # Keep the boxes with IoU less than thresh_iou
        mask = IoU < threshIoU
        order = order[mask]
        
    return keep

In [6]:
P = torch.tensor([
    [1, 1, 3, 3, 0.95],
    [1, 1, 3, 4, 0.93],
    [1, 0.9, 3.6, 3, 0.98],
    [1, 0.9, 3.5, 3, 0.97]
])

In [12]:
filteredBoxes = []
print(filteredBoxes)

[]


In [10]:
filteredBoxes = nms_pytorch(P, 0.5)
print(filteredBoxes)

[tensor([1.0000, 0.9000, 3.6000, 3.0000, 0.9800])]


In [13]:
filteredBoxes = nms_pytorch(P, 0.8)
print(filteredBoxes)

[tensor([1.0000, 0.9000, 3.6000, 3.0000, 0.9800]), tensor([1.0000, 1.0000, 3.0000, 3.0000, 0.9500]), tensor([1.0000, 1.0000, 3.0000, 4.0000, 0.9300])]
